#Análisis de sentimiento como clasificación de textos

Objetivo: Clasificar reseñas de películas como aquellas que valoran de forma positiva o negativa una película


Ejemplo sencillo de hacer análisis de la opinión como una tarea de clasificación o de categorización de textos. 

Usaremos el corpus que se uso en uno de los primeros estudios acerca de análisis de sentimiento (movie_reviews) (Pang and Lee 2002) 

In [ ]:
import nltk

from nltk.corpus import movie_reviews

print movie_reviews.readme()

## Inspección del corpus

In [ ]:
# Mostrar las categorias: clasificación binaria entre textos positivos y negativos
movie_reviews.categories()

In [ ]:
## Obtener los identificadores de fichero - podemos ver los datos en el directorio nltk_data 
## o inspeccionar mediante las utilidades de la clase CorpusReader

files = movie_reviews.fileids()

In [ ]:
files[100]  ## Nombre del fichero con el id = 1000 - la carpeta indica la categoría
## Los ficheros con cada review se representan como una lista

In [ ]:
movie_reviews.categories(files[0]) # ¿Cual es la categoría del documento? 
## categories(list) acepta una lista de ids de documentos y obtinene el conjunto de categorias

In [ ]:
movie_reviews.fileids('pos') # Obten todos los documentos etiquetados como una categorias

In [ ]:
## ¿Cuantos ejemplos hay de cada una de las clases? 

print "Ejemplos positivos: %s" % len(movie_reviews.fileids('pos'))
print "Ejemplos negativos: %s" % len(movie_reviews.fileids('neg'))


## Inspeccionar algun documento

In [ ]:
doc = movie_reviews.words(files[0]) # Recordad que la representación por defecto de nltk es como una lista de tokens

print " ".join(doc)

Inspecciona varios documentos y trata de leer las reviews:
 * ¿Que características tiene el texto respecto a tokenización? 
 * Identifica que tipo de características podrían ser interesantes para la tareas 
 * ¿Son generalizables? 
 * Identifica problemas, pero sin quedarse en lo anecdótico

## Inspeccionar las características del corpus

In [ ]:
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
all_words.most_common() # Mostrar la lista de palabras más comunes

## Entrenar y evaluar un Clasificador Naive Bayes

In [ ]:
import nltk.classify.util

from nltk.classify import NaiveBayesClassifier

### Seleccion de características: Todas las palabras

Inicialmente seleccionamos todos los tokens de un texto como características de nuestro clasificador

In [ ]:
# Implementamos la extraccion de características como una función que se aplica a la lista de tokens de un documento
# NLTK nos permite representar las caracxterísticas como un diccionario

def word_feats(words):
    return dict([(word, True) for word in words])

### Generación de los conjuntos de datos

In [ ]:
# Conjunto de instancias - documento
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

# Vectores de características
negfeatures = [(word_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
posfeatures = [(word_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]

# Utilizamos 
#  - 3/4 partes de los documentos de cada clase para entrenamiento
#  - 1/4 part para evaluación 
negcutoff = len(negfeatures)*3/4
poscutoff = len(posfeatures)*3/4

trainfeats = negfeatures[:negcutoff] + posfeatures[:poscutoff]
testfeats = negfeatures[negcutoff:] + posfeatures[poscutoff:]
print 'train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats))


In [ ]:
 ¿Qué pinta tienen los vectores de características?

¿Qué pinta tienen los vectores de características

In [ ]:
print trainfeats[100]

## Entrenamiento del clasificador

In [ ]:
# Usamos el clasificador NaiveBayes de NLTK - Bernouilli NB (binario)

classifier = NaiveBayesClassifier.train(trainfeats)

## Evaluación del clasificador (1)

Como primera aproximación usamos la medida de **Accuracy** - Porcentaje de acierto

### Discusion
* ¿Es adecuada para este dataset? 
* ¿Es adecuada para la tarea de análisis de sentimiento? 
* ¿Y si tuviesemos varias clases de sentimiento: positivo, negativo, neutro?
* ¿Y si los ejemplos estuviesen desbalancesados?

In [ ]:
print 'accuracy:', nltk.classify.util.accuracy(classifier, testfeats)

## Mostrar las características más informativas

In [ ]:
classifier.show_most_informative_features(50)

## Evaluación del clasificador (2)

Otras medidas de evaluación: Precision, Recall (Cobertura) y su combinación la medida F

\begin{equation*}
Precision = \frac{tp}{tp +fp}
\end{equation*}

\begin{equation*}
Recall = \frac{tp}{tp +fn}
\end{equation*}

F es la media armónica entre la precision y la cobertura

\begin{equation*}
F = 2 · \frac{Precision · Recall}{Precision + Recall}
\end{equation*}


In [ ]:
import collections

refsets = collections.defaultdict(set)
testsets = collections.defaultdict(set)


for i, (feats, label) in enumerate(testfeats):
        refsets[label].add(i)
        observed = classifier.classify(feats)
        testsets[observed].add(i)
 
print 'pos precision:', nltk.metrics.precision(refsets['pos'], testsets['pos'])
print 'pos recall:', nltk.metrics.recall(refsets['pos'], testsets['pos'])
print 'pos F-measure:', nltk.metrics.f_measure(refsets['pos'], testsets['pos'])
print 'neg precision:', nltk.metrics.precision(refsets['neg'], testsets['neg'])
print 'neg recall:', nltk.metrics.recall(refsets['neg'], testsets['neg'])
print 'neg F-measure:', nltk.metrics.f_measure(refsets['neg'], testsets['neg'])

## Discusión

* ¿Resultados?
* ¿Que se puede mejorar? 
 * Construccion del conjunto de entrenamiento/test 
 * Selección de características
 * Procesimiento para elegir características - ¿estamos viendo las características de test?
 * Algoritmos de aprendizaje